# Introduction

This vignette introduces loss model in GEMAct package.
The notebook shows how to compute the aggregate loss distribution of different combinations of policy structures and underlying frequency and severity models. Different approaches are offered, namely Panjer Recursion, Fast Fourier Transform and Monte Carlo methods.

For additional details on loss model functionalities and methods visit [website](https://gem-analytics.github.io/gemact/api.html#lossmodel). 

In [ ]:
pip install gemact

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.1 MB/s 
  Attempting uninstall: gemact
    Found existing installation: gemact 0.1.3
    Uninstalling gemact-0.1.3:
      Successfully uninstalled gemact-0.1.3


In [ ]:
import gemact
import numpy as np

# Computation of the aggregate loss

Within the framework of the collective risk model [1] the aggregate loss $X$ is modeled as

$$
X=\sum_{i=1}^{N} Z_i,
$$

where $N$ is the random number of claims (frequency) and $Z_i,\; i=1,2,...,N$ is the payment amount for the $i$-th claim (severity). The underlying assumptions are the following:

* $Z_i$ are independent and identically distributed (iid) random variables. 
* $N$ and $Z_i$ are independent.

In [ ]:
# define a Poisson frequency model
frequency = Frequency(
    dist='poisson',
    par={'mu': 4}
    )

# define a Generalized Pareto severity model
severity = Severity(
    dist='genpareto',
    par={'c': .2, 'scale': 1}
    )

# define PolicyStructure (default is a empty policy, namely infinty-xs-0 XL with no Quota Share).
policystructure = PolicyStructure()

# create LossModel with above defined objects.
# aggr_loss_dist_method approach set to 'mc', i.e. Monte Carlo.
lossmodel_mc = LossModel(
    frequency=frequency,
    severity=severity,
    policystructure=policystructure,
    aggr_loss_dist_method='mc',
    n_sim=1e+05,
    random_state=1
    )

INFO:lossmodel|..Approximating aggregate loss distribution via Monte Carlo simulation..
INFO:lossmodel|..Simulation completed..


In [ ]:
# create LossModel with above defined objects.
# aggr_loss_dist_method approach is here set to 'recursion', i.e. Panjer Recursion.
lossmodel_rec = LossModel(
    frequency=frequency,
    severity=severity,
    policystructure=policystructure,
    aggr_loss_dist_method='recursion',
    n_sev_discr_nodes=int(1000),
    sev_discr_step=1,
    n_aggr_dist_nodes=int(10000)
    )

INFO:lossmodel|..Approximating aggregate loss distribution via Panjer recursion..
INFO:lossmodel|..Panjer recursion completed..


In [ ]:
# create LossModel with above defined objects.
# finally, aggr_loss_dist_method approach is set to 'fft', i.e. Fast Fourier Transform.
lossmodel_dft = LossModel(
    frequency=frequency,
    severity=severity,
    policystructure=policystructure,
    aggr_loss_dist_method='fft',
    n_sev_discr_nodes=int(1000),
    sev_discr_step=1,
    n_aggr_dist_nodes=int(10000)
    )

INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


In [ ]:
# print averages of the aggregate loss distribution for the three LossModels.
print('MC: ',lossmodel_mc.aggr_loss_mean())
print('RECURSIVE: ',lossmodel_rec.aggr_loss_mean())
print('FFT: ',lossmodel_dft.aggr_loss_mean())

MC 4.986130567647585
RECURSIVE 4.999999984652499
FFT 4.9999999846531935


# (Re)insurance contract costing

Below the costing of the following reinsurance covers is presented: 

* Excess-of-loss (XL),
* Excess-of-loss with aggregate conditions (XLagg),
* Excess-of-loss with reinstatements (RS).
* Stop Loss (SL) with different partecipation shares.

In [ ]:
# severity, frequency and policystructure can be created on-the-fly within LossModel definition.
# PolicyStructure has a 100-xs-100 XL Layer.
lossmodel_XL = LossModel(
    frequency=Frequency(
        dist='poisson',
        par={'mu': .5}
    ),
    severity=Severity(
        par= {'loc': 0,
        'scale': 83.34,
        'c': 0.834},
        dist='genpareto'
    ),
    policystructure=PolicyStructure(
        layers=Layer(
            cover=100,
            deductible=100
        )
    ),
    aggr_loss_dist_method='fft',
    sev_discr_method='massdispersal',
    n_sev_discr_nodes=int(10000),
    sev_discr_step=.01,
    n_aggr_dist_nodes=int(100000)
    )

INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


One can retrieve information about the policy layer conditions and the aggregate loss distribution approximatiion method by adopting the following methods of the lossmodel object.

In [ ]:
lossmodel_XL.print_policy_layer_specs()
lossmodel_XL.print_aggr_loss_method_specs()

In addition, also costing information can be printed as shown below.

In [ ]:
lossmodel_XL.print_costing_specs()

                     Contract specification            parameter                value
                               Deductible                    d                100.0
                                    Cover                u - d                100.0
                           Upper priority                    u                200.0
                     Aggregate deductible                    L                100.0
                     Quota share ceded portion                alpha                    1
                             Pure premium                    P   1.3354758296850144

 Reinstatement layer loading c:  []
fft 	 n_sev_discr_nodes m:  9999 	 n_aggr_dist_nodes n:  100000


# Excess-of-loss with aggregate conditions

Remark: as shown in the example below, a LossModel object can be instanciated without directly specifying the approach to calculate the aggregate loss distribution (aggr_loss_dist_method). This can be set later, before executing the method to calculate the aggregate loss distribution (aggr_loss_dist_calculate).

In [ ]:
# PolicyStructure has a 50-xs-50 XL Layer, 50 aggregate deductible, 150 aggregate limit.
lossmodel_XLagg = LossModel(
    frequency=Frequency(
        dist='poisson',
        par={'mu': .5}
    ),
    severity=Severity(
        par={'loc': 0,
        'scale': 83.34,
        'c': 0.834},
        dist='genpareto'
    ),
    policystructure=PolicyStructure(
        layers=Layer(
            cover=50,
            deductible=50,
            aggr_deductible=100,
            aggr_cover=150
            )
        )
    )

# option 1: use the aggr_loss_dist_calculate with its arguments.
lossmodel_XLagg.aggr_loss_dist_calculate(
    aggr_loss_dist_method='fft',
    sev_discr_method='massdispersal',
    n_sev_discr_nodes=10000,
    sev_discr_step=.01,
    n_aggr_dist_nodes=100000
)
# print the resulting aggregate loss distribution.
aggr_loss_dist_a = lossmodel_XLagg.aggr_loss_dist[0]

# option 2: use the aggr_loss_dist_calculate after having set the property values.
lossmodel_XLagg.aggr_loss_dist_method='fft'
lossmodel_XLagg.sev_discr_method='massdispersal'
lossmodel_XLagg.n_sev_discr_nodes=10000
lossmodel_XLagg.sev_discr_step=.01
lossmodel_XLagg.n_aggr_dist_nodes=100000
lossmodel_XLagg.aggr_loss_dist_calculate()
# print the resulting aggregate loss distribution.
aggr_loss_dist_b = lossmodel_XLagg.aggr_loss_dist[0]

# check equality
aggr_loss_dist_b == aggr_loss_dist_a

In [ ]:
# run the risk costing method for the XL with aggregate conditions
lossmodel_XLagg.costing()

# Excess-of-loss with reinstatements

In [ ]:
# PolicyStructure has a 100-xs-0 XL Layer, 0 aggregate deductible, 1 reinstatement and 50% reinstatement loading.
# aggr_loss_dist_method is 'fft', namely Fast Fourier Transformation.
# sev_discr_method is 'massdispersal', namely mass dispersal method.
lossmodel_RS = LossModel(
    frequency=Frequency(
        dist='poisson',
        par={'mu': .5}
    ),
    severity=Severity(
        par= {'loc': 0,
              'scale': 83.34,
              'c': 0.834},
        dist='genpareto'
    ),
    policystructure=PolicyStructure(
        layers=Layer(
            cover=100,
            deductible=0,
            aggr_deductible=0,
            reinst_loading=0.5,
            n_reinst=1
        )
    ),
    aggr_loss_dist_method='fft',
    sev_discr_method='massdispersal',
    n_sev_discr_nodes=int(10000),
    sev_discr_step=.01,
    n_aggr_dist_nodes=int(100000)
    )

INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


In [ ]:
lossmodel_RS.costing()

                     Contract specification            parameter                value
                               Deductible                    d                  0.0
                                    Cover                u - d                100.0
                           Upper priority                    u                100.0
                     Aggregate deductible                    L                  0.0
                     Quota share ceded portion                alpha                    1
                     Number of reinstatements                    K                    1
                             Pure premium                    P    24.97739773475006

 Reinstatement layer loading c:  [1]
fft 	 n_sev_discr_nodes m:  9999 	 n_aggr_dist_nodes n:  100000


# Stop Loss

Remark: as shown in the example below, a policystructure can contain multiple layers (and/or shares) and be applied to a single frequency-severity model in a single LossModel object.
PolicyStructure layers and shares can be expressed as lists, whose items are associated accordingly (i.e. first item of layers list with first item of shares list, etc.).

In [ ]:
frequency = Frequency(dist='nbinom', par={'n': 100, 'p': .06})
severity = Severity(
    par= {'loc': 0,
    'scale': 83.34,
    'c': 0.834},
    dist='genpareto'
    )
policystructure = PolicyStructure(
        layers=[
            Layer(aggr_deductible=100, aggr_cover=200),
            Layer(aggr_deductible=75, aggr_cover=175)
            ],
        shares=[0.75, 0.65]
        )

lossmodel_SL = LossModel(
    frequency=frequency,
    severity=severity,
    policystructure=policystructure
    )

# using the aggr_loss_dist_calculate with its arguments.
lossmodel_SL.aggr_loss_dist_calculate(
    aggr_loss_dist_method='mc',
    n_sim=1e+05,
    random_state=1
)
# the resulting aggregate loss distribution is a list with two entries,
# one for each layer of the policystructure.
lossmodel_SL.aggr_loss_dist[0]
lossmodel_SL.aggr_loss_dist[1]

In [ ]:
# costing of the first policystructure layer, namely 200-xs-100 SL with 75% partecipation.
lossmodel_SL.costing(idx=0)
# costing of the first policystructure layer, namely 175-xs-75 SL with 65% partecipation.
lossmodel_SL.costing(idx=1)

# Severity discretization

In order to move from a continuous distribution to an dicretized distribution, it is important to preserve certain distribution properties and statistics either locally or globally. Given a span $h$ and the last available point $M$, in GEMAct there are two available solutions.

**Method of mass dispersal**

$$ f_{0}=\operatorname{Pr}\left(\widetilde{Y}<\frac{h}{2}\right)=F_{\widetilde{Y}}\left(\frac{h}{2}-0\right)$$

$$ f_{j}=F_{\widetilde{Y}}\left(j h+\frac{h}{2}-0\right)-F_{\widetilde{Y}}\left(j h-\frac{h}{2}-0\right), \quad j=1,2, \ldots, M-1$$


$$f_{M}=1-F_{X}[(M-0.5) h-0]$$

**Method of local moments matching**

The following approach is applied to preserve the global mean of the distribution.

$$f_0 = m^0_0$$

$$f_j = m^{j}_0+ m^{j-1}_1 , \quad j=0,1, \ldots, M$$

$$ \sum_{j=0}^{1}\left(x_{k}+j h\right)^{r} m_{j}^{k}=\int_{x_{k}-0}^{x_{k}+ h-0} x^{r} d F_{X}(x), \quad r=0,1$$

$$ m_{j}^{k}=\int_{x_{k}-0}^{x_{k}+p h-0} \prod_{i \neq j} \frac{x-x_{k}-i h}{(j-i) h} d F_{X}(x), \quad j=0,1$$


Below the computation consistency is shown by comparing the original mean with the discretized distribution means obtained via mass dispersal and local moments.

In [ ]:
# severity objects are equipped with a method for performing the discretization.
# this can be carried out using either 'massdispersal' or 'localmoments' argument.

massdispersal = severity.discretize(
    discr_method='massdispersal',
    n_discr_nodes=50000,
    discr_step=.01,
    deductible=0,
    cover=100
)

localmoments = severity.discretize(
    discr_method='localmoments',
    n_discr_nodes=50000,
    discr_step=.01,
    deductible=0,
    cover=100
)

meanMD = np.sum(massdispersal['sev_nodes'] * massdispersal['fj'])
meanLM = np.sum(localmoments['sev_nodes'] * localmoments['fj'])
print('Original mean: ', severity.model.mean())
print('Mean (mass dispersal): ', meanMD)
print('Mean (local moments): ', meanLM)

INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..
INFO:lossmodel|Discretization step set to cover/n_sev_discr_nodes.
INFO:lossmodel|..Approximating aggregate loss distribution via FFT..
INFO:lossmodel|..Distribution via FFT completed..


1. Klugman S.A., Panjer  H.H., and Willmot G.E. "Loss models: from data to decisions", 2012, John Wiley & Sons.